## System Description
1. We have a set of COFs from a database. Each COF is characterized by a feature vector $$x_{COF} \in X \subset R^d$$ were d=14.


2. We have **two different types** of simulations to calculate **the same material property $S_{Xe/Kr}$**. Therefore, we have a Single-Task/Objective 
$$argmax_{x_{COF} \in X}[S_{Xe/Kr}(x_{COF})]$$

3. Multi-Fidelity problem. 
    1. low-fidelity  => Henry coefficient calculation - MC integration: $S_{Xe/Kr} = \frac{H_{Xe}}{H_{Kr}}$
    2. high-fidelity => GCMC mixture simulation - 80:20 (Kr:Xe) at 298 K and 1.0 bar: $S_{Xe/Kr} = \frac{n_{Xe} / n_{Kr}}{y_{Xe}/y_{Kr}}$


3. We will initialize the system with a few COFs at **both** fidelities in order to initialize the Covariance Matrix.
    1. The fist COF will be the one closest to the center of the normalized feature space
    2. The rest will be chosen to maximize diversity of the training set


4. Each surrogate model will **only train on data acquired at its level of fidelity** (Heterotopic data). $$X_{lf} \neq X_{hf} \subset X$$
    1. We  use the augmented-EI (aEI) acquisition function from [here](https://link.springer.com/content/pdf/10.1007/s00158-005-0587-0.pdf)
    2. Botorch GP surrogate model: [SingleTaskMultiFidelityGP](https://botorch.org/api/models.html#module-botorch.models.gp_regression_fidelity)
    3. Needed to use [this](https://botorch.org/api/optim.html#module-botorch.optim.fit) optimizer to correct matrix jitter
    4. Helpful [tutorial](https://botorch.org/tutorials/discrete_multi_fidelity_bo) for a similar BoTorch Model used

In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskMultiFidelityGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py # for .jld2 files
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

## Load Data

In [2]:
normalization = "min_max_normalization" # "standardization"

In [3]:
###
#  load data: targets and features
###
if normalization == 'min_max_normalization':
    file = h5py.File("targets_and_normalized_features.jld2", "r")
else:
    file = h5py.File("targets_and_standardized_features.jld2", "r")

# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data
y = [torch.from_numpy(np.transpose(file["henry_y"][:])), 
     torch.from_numpy(np.transpose(file["gcmc_y"][:]))]
# associated simulation costs
cost = [np.transpose(file["henry_total_elapsed_time"][:]), 
        np.transpose(file["gcmc_elapsed_time"][:])]

# total number of COFs in data set
nb_COFs = X.shape[0] 

In [4]:
###
#  load data: initializing COFs
###
init_cof_ids_file = pickle.load(open('search_results/{}/initializing_cof_ids.pkl'.format(normalization), 'rb'))
init_cof_ids = init_cof_ids_file['init_cof_ids']

# total number of BO searches to run = number of initializing sets
nb_runs = len(init_cof_ids)

print("raw data - \n\tX:", X.shape)
for f in range(2):
    print("\tfidelity:", f)
    print("\t\ty:", y[f].shape)
    print("\t\tcost: ", cost[f].shape)
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)
print("width:\n",torch.max(X, 0).values - torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
	fidelity: 0
		y: torch.Size([608])
		cost:  (608,)
	fidelity: 1
		y: torch.Size([608])
		cost:  (608,)

Ensure features are normalized - 
max:
 tensor([5.6399e+01, 9.2784e-01, 6.3570e+03, 1.6107e+03, 1.8182e-01, 2.5000e-01,
        6.6667e-01, 5.0000e-01, 2.9520e-02, 3.3333e-01, 1.4286e-01, 6.6667e-02,
        2.8571e-01, 2.3810e-02], dtype=torch.float64)
min:
 tensor([3.5094e+00, 1.6356e-01, 1.9966e+03, 1.0272e+02, 0.0000e+00, 0.0000e+00,
        3.2500e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], dtype=torch.float64)
width:
 tensor([5.2889e+01, 7.6428e-01, 4.3604e+03, 1.5080e+03, 1.8182e-01, 2.5000e-01,
        3.4167e-01, 5.0000e-01, 2.9520e-02, 3.3333e-01, 1.4286e-01, 6.6667e-02,
        2.8571e-01, 2.3810e-02], dtype=torch.float64)


# TODO:

### from botorch.models.transforms.input import Normalize
### X_tf = X.Normalize()


okay, so I think that the immediate plan is going to have to be to generate a file of the raw feature vectors
and pass those into the model while using the `input_transform=Normalize(d=?)` or the `input_transform=Standardize(d=?)` where `d = the dimension of the input space`. 

- [normalization](https://botorch.org/api/_modules/botorch/models/transforms/input.html#Normalize)
- [standardization](https://botorch.org/api/_modules/botorch/models/transforms/input.html#InputStandardize)


I will have to be sure to save the output of these models in files deparate from those of the BO runs that I have already made. This way I can avoid any issues with overwriting the data and wasting time.

Be certain that everything that gets done here happens for the SFBO!

**NOTE:**
Also, leave notes and links to papers and documentation wherever I can to help Cory in the writing process. This will also help the reader when going through the notebooks/project. For example, the optimizer used by the GP as well as which kernel function it is using.

Oh, and push a link (i.e. download a copy of the file as a ptx) to the MFBO_Fig1 powerpoint so Cory has access to it as well.

In [5]:
# print("total high-fidelity cost:", sum(cost[1]).item(), "[min]")
# print("total low-fidelity cost: ", sum(cost[0]).item(), "[min]\n")

# print("average high-fidelity cost:", np.mean(cost[1]), "[min]")
# print("average low-fidelity cost: ", np.mean(cost[0]), "[min]")
# print("average cost ratio:\t   ", np.mean(cost[1] / cost[0]))

total high-fidelity cost: 139887.66223703226 [min]
total low-fidelity cost:  10076.305239888028 [min]

average high-fidelity cost: 230.0783918372241 [min]
average low-fidelity cost:  16.57287046034216 [min]
average cost ratio:	    13.444745568580501


## Helper Functions

#### Post-Search Analysis

In [6]:
def calc_fidelity_fraction(acquired_set):
    nb_iters = len(acquired_set)
    fid_frac = np.zeros(nb_iters)
    for i in range(1, nb_iters):
        fid_frac[i] = sum(acquired_set[:, 0][:i+1]) / (i+1)
    return fid_frac

In [7]:
def get_y_maxes_acquired(acquired_set):    
    nb_iters = len(acquired_set)
    y_maxes = np.zeros(nb_iters)
    # we want the maximum y value (only high-fidelity) up to a given iteration
    y_max = 0.0 # update this each iteration.
    for i, (f_id, cof_id) in enumerate(acquired_set):
        y_acq_this_iter = y[f_id][cof_id]
        # i is iteration index
        if f_id == 1 and y_acq_this_iter > y_max:
            y_max = y_acq_this_iter # over-write max
        y_maxes[i] = y_max 
    return y_maxes

In [8]:
def accumulated_cost(cost_acquired):
    nb_iters = len(acquired_set)
    accumulated_cost = np.zeros(nb_iters)
    accumulated_cost[0] = cost_acquired[0]
    for i in range(1, len(cost_acquired)):
        accumulated_cost[i] = accumulated_cost[i-1] + cost_acquired[i]
    return accumulated_cost

#### Construct Initial Inputs

In [9]:
def initialize_acquired_set(X, y, initializing_COFs, discrete_fidelities):
#     cof_ids = diverse_set(X, nb_COFs_initialization) # np.array(ids_train)
    return torch.tensor([[f_id, cof_id] for cof_id in initializing_COFs for f_id in discrete_fidelities])

In [10]:
# construct feature matrix of acquired points
def build_X_train(acquired_set):
    cof_ids = [a[1] for a in acquired_set]
    f_ids = torch.tensor([a[0] for a in acquired_set])
    return torch.cat((X[cof_ids, :], f_ids.unsqueeze(dim=-1)), dim=1)

# construct output vector for acquired points
def build_y_train(acquired_set):
    return torch.tensor([y[f_id][cof_id] for f_id, cof_id in acquired_set]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(acquired_set):
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in acquired_set]).unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost_fidelity(acquired_set, fidelity):
    return torch.tensor([cost[f_id][cof_id] for f_id, cof_id in acquired_set if f_id == fidelity]).unsqueeze(-1)

#### test

In [11]:
def test_initializing_functions(X, y):
    ###
    #  construct training sets
    ###
    # list of (cof_id, fid_id)'s
    acquired_set = [[1, 10], [0, 3], [0, 4]]
    
    # Training Sets
    X_train = build_X_train(acquired_set)
    y_train = build_y_train(acquired_set)
    
    ###
    #  test that the constructor functions are working properly
    ###
    assert np.allclose(X[10, :], X_train[0, :14])
    assert X_train[0, 14] == 1
    assert X_train[1, 14] == 0
    assert y_train[0] == y[1][10] # y[fid_id][cof_id]
    assert y_train[2] == y[0][4]
    return

test_initializing_functions(X, y)

### Surrogate Model

In [12]:
def train_surrogate_model(X_train, y_train):
    model = SingleTaskMultiFidelityGP(
        X_train, 
        y_train, 
        outcome_transform=Standardize(m=1), # m is the output dimension
        data_fidelity=X_train.shape[1] - 1
    )   
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

### Acquisition Function

In [13]:
# calculate posterior mean and variance for a given fidelity
def mu_sigma(model, X, fidelity):
    f = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    f_posterior = model.posterior(X_f)
    return f_posterior.mean.squeeze().detach().numpy(), np.sqrt(f_posterior.variance.squeeze().detach().numpy())

# get the current best y-value of desired_fidelity in the acquired set
def get_y_max(acquired_set, desired_fidelity):
    return np.max([y[f_id][cof_id] for f_id, cof_id in acquired_set if f_id == desired_fidelity])

In [14]:
###
#  efficient multi-fidelity correlation function
#  corr(y at given fidelity, y at high-fidelity)
#  (see notes)
###
def mfbo_correlation_function(model, X, fidelity):
    # given fidelity
    f   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) * fidelity
    X_f = torch.cat((X, f), dim=1) # last col is associated fidelity
    
    #  high-fidelity
    hf   = torch.tensor((), dtype=torch.float64).new_ones((nb_COFs, 1)) 
    X_hf = torch.cat((X, hf), dim=1) # last col is associated fidelity

    # combine into a single tensor
    X_all_fid = torch.cat((X_f, X_hf), dim=0)
    
    # get variance for each fidelity
    var_f = torch.flatten(model.posterior(X_f).variance)
    var_hf = torch.flatten(model.posterior(X_hf).variance) # variance
    
    # posterior covariance 
    cov = torch.diag(model(X_all_fid).covariance_matrix[:X_f.size()[0], X_f.size()[0]:])
    
    corr = cov / (torch.sqrt(var_f) * torch.sqrt(var_hf))
    return corr

In [15]:
###
#  cost ratio
###
def estimate_cost_ratio(fidelity, acquired_set):
    avg_cost_f  = torch.mean(build_cost_fidelity(acquired_set, fidelity))
    avg_cost_hf = torch.mean(build_cost_fidelity(acquired_set, 1))
    cr = avg_cost_hf / avg_cost_f
    return cr.item()

###
#  expected imrovement function, only uses hf
###
def EI_hf(model, X, acquired_set):
    hf_mu, hf_sigma = mu_sigma(model, X, 1)
    y_max = get_y_max(acquired_set, 1)
    
    z = (hf_mu - y_max) / hf_sigma
    explore_term = hf_sigma * norm.pdf(z) 
    exploit_term = (hf_mu - y_max) * norm.cdf(z) 
    ei = explore_term + exploit_term
    return np.maximum(ei, np.zeros(nb_COFs))

###
#  acquisition function
###
def acquisition_scores(model, X, fidelity, acquired_set):
    # expected improvement for high-fidelity
    ei = EI_hf(model, X, acquired_set) 
    
    # augmenting functions
    corr_f1_f0 = mfbo_correlation_function(model, X, fidelity)
    
    cr = estimate_cost_ratio(fidelity, acquired_set)

    scores = torch.from_numpy(ei) * corr_f1_f0 * cr
    return scores.detach().numpy()

def in_acquired_set(f_id, cof_id, acquired_set):
    for this_f_id, this_cof_id in acquired_set:
        if this_cof_id == cof_id and this_f_id == f_id:
            return True
    return False

### Bayesian Algorithm

In [16]:
def run_Bayesian_optimization(nb_iterations, initializing_COFs, verbose=False):
    assert nb_iterations > len(initializing_COFs)
    ###
    #  initialize system
    ###
    acquired_set = initialize_acquired_set(X, y, initializing_COFs, discrete_fidelities)
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(nb_COFs_initialization * len(discrete_fidelities), nb_iterations): 
        print("BO iteration: ", i)
        ###
        #  construct training data (perform experiments)
        ###
        X_train = build_X_train(acquired_set)
        y_train = build_y_train(acquired_set)

        if verbose:
            print("Initialization - \n")
            print("\tid acquired = ", [acq_[0].item() for acq_ in acquired_set])
            print("\tfidelity acquired = ", [acq_[1].item() for acq_ in acquired_set])
            print("\tcosts acquired = ", build_cost(acquired_set), " [min]")

            print("\n\tTraining data:\n")
            print("\t\t X train shape = ", X_train.shape)
            print("\t\t y train shape = ", y_train.shape)
            print("\t\t training feature vector = \n", X_train)
        
        ###
        #  train Model
        ###
        model = train_surrogate_model(X_train, y_train)

        ###
        #  acquire new (COF, fidelity) not yet acquired.
        ###
        # entry (fid_id, cof_id) is the acquisition value for fidelity f_id and cof cof_id
        the_acquisition_scores = np.array([acquisition_scores(model, X, f_id, acquired_set) for f_id in discrete_fidelities])
        # overwrite acquired COFs/fidelities with negative infinity to not choose these.
        for f_id, cof_id in acquired_set:
            the_acquisition_scores[f_id, cof_id] = - np.inf
        # select COF/fidelity with highest aquisition score.
        f_id, cof_id = np.unravel_index(np.argmax(the_acquisition_scores), np.shape(the_acquisition_scores))
        assert not in_acquired_set(f_id, cof_id, acquired_set)
        # update acquired_set
        acq = torch.tensor([[f_id, cof_id]], dtype=int)
        acquired_set = torch.cat((acquired_set, acq))

        ###
        #  print useful info
        ###
        if verbose:
            print("\tacquired COF ", cof_id, " at fidelity, ", f_id)
            print("\t\ty = ", y[f_id][cof_id].item())
            print("\t\tcost = ", cost[f_id][cof_id])
        
    return acquired_set

# Run MFBO

In [17]:
###
#  construct initial inputs
###
discrete_fidelities = [0, 1] # set of discrete fidelities to select from
nb_COFs_initialization = 3   # at each fidelity, number of COFs to initialize with
nb_iterations = 150          # BO budget, includes initializing COFs

In [18]:
###
#  run search
###
for j, initializing_COFs in enumerate(init_cof_ids): 
    # check the length of each initializing set
    assert len(initializing_COFs) == nb_COFs_initialization
    print("run #: {}".format(j))
    
    # start timer for BO run
    start_time = time.time()
    ###
    #  run BO search
    ###
    acquired_set = run_Bayesian_optimization(nb_iterations, initializing_COFs)
    
    ###
    #  post-run analysis
    ###
    elapsed_time  = time.time() - start_time
    print("elapsed_time:\t", elapsed_time / 60, " min.")
    y_acquired    = build_y_train(acquired_set)
    y_maxes_acq   = get_y_maxes_acquired(acquired_set.detach().numpy())
    fid_fraction  = calc_fidelity_fraction(acquired_set.detach().numpy())
    cost_acquired = build_cost(acquired_set)
    acc_cost      = accumulated_cost(cost_acquired.flatten().detach().numpy())
    
    ###
    # look at unique COFs acquired
    ###
    # cof_ids_acquired = torch.tensor([acq[1] for acq in acquired_set])
    n_unique_cofs_acquired = len(np.unique([acq[1] for acq in acquired_set]))
    print("total number of unique COFs acquired", n_unique_cofs_acquired)

    ###
    #  Iterations until top COF and accumulated 
    ###
    cof_id_with_max_selectivity = np.argmax(y[1])
    BO_iter_top_cof_acquired = float("inf") # dummy 
    for i, (f_id, cof_id) in enumerate(acquired_set):
        if cof_id == cof_id_with_max_selectivity and f_id == 1:
            BO_iter_top_cof_acquired = i
            print("woo, top COF acquired!")
            print("iteration we acquire top COF = ", BO_iter_top_cof_acquired) 
            break
        elif i == len(acquired_set)-1:
            print("oh no, top COF not acquired!")


    top_cof_acc_cost = sum(build_cost(acquired_set)[:BO_iter_top_cof_acquired])
    print("accumulated cost up to observation of top COF = ", top_cof_acc_cost.item(), " [min]")
    
    ###
    #  store results
    ###
    mfbo_res = dict({'acquired_set': acquired_set.detach().numpy(),
                     'y_acquired': y_acquired.detach().numpy(),
                     'y_max_acquired': y_maxes_acq,
                     'fidelity_fraction': fid_fraction,
                     'cost_acquired': cost_acquired.flatten().detach().numpy(),
                     'accumulated_cost': acc_cost / 60,
                     'nb_COFs_initialization': nb_COFs_initialization,
                     'BO_iter_top_cof_acquired': BO_iter_top_cof_acquired,
                     'elapsed_time (min)':  elapsed_time / 60
                    })

    with open('search_results/{}/mfbo_results/mfbo_results_run_{}.pkl'.format(normalization, j), 'wb') as file:
        pickle.dump(mfbo_res, file)

run #: 0
BO iteration:  6
Iter 10/100: 8.650376765494741
Iter 20/100: 6.264916015862133
Iter 30/100: 5.001508348167349
Iter 40/100: 4.227785214725494
Iter 50/100: 3.9908897963763903
Iter 60/100: 3.882949650779772
Iter 70/100: 3.8390306004209656
Iter 80/100: 3.819243406045588
Iter 90/100: 3.8110482061898288
Iter 100/100: 3.806019874799117
BO iteration:  7
Iter 10/100: 7.651159008279277
Iter 20/100: 5.591460754291136
Iter 30/100: 4.49198211766518
Iter 40/100: 3.825042688141902
Iter 50/100: 3.564799857926629
Iter 60/100: 3.4443282339104644
Iter 70/100: 3.4003119062128917
Iter 80/100: 3.3835044555142035
Iter 90/100: 3.3781348813755927
Iter 100/100: 3.3765287721455723
BO iteration:  8
Iter 10/100: 6.898288290928619
Iter 20/100: 5.08665614038297
Iter 30/100: 4.114948187571303
Iter 40/100: 3.5211723404038318
Iter 50/100: 3.3472862138199027
Iter 60/100: 3.270972160000066
Iter 70/100: 3.238273122021649
Iter 80/100: 3.2232895573824196
Iter 90/100: 3.2164071293457015
Iter 100/100: 3.2117380488239

Iter 100/100: 1.8665331152849496
BO iteration:  31
Iter 10/100: 2.9507640681670697
Iter 20/100: 2.40954334824012
Iter 30/100: 2.1207891452901166
Iter 40/100: 1.9915037062733991
Iter 50/100: 1.90350046689949
Iter 60/100: 1.8683967212465193
Iter 70/100: 1.8585822020518563
Iter 80/100: 1.854373380856268
Iter 90/100: 1.853095089093779
Iter 100/100: 1.852532671698425
BO iteration:  32
Iter 10/100: 2.9077323988998756
Iter 20/100: 2.3800911885189047
Iter 30/100: 2.0982692471775684
Iter 40/100: 1.9738786820788712
Iter 50/100: 1.8889003758506104
Iter 60/100: 1.8573369249665213
Iter 70/100: 1.8449086445187222
Iter 80/100: 1.840079088223956
Iter 90/100: 1.838951076041133
Iter 100/100: 1.838463192920785
BO iteration:  33
Iter 10/100: 2.8677957664787894
Iter 20/100: 2.3531571731293717
Iter 30/100: 2.0785833552325235
Iter 40/100: 1.959240796616318
Iter 50/100: 1.8773198570907512
Iter 60/100: 1.843537936869335
Iter 70/100: 1.8328257306468179
Iter 80/100: 1.8279764500480897
Iter 90/100: 1.826920572666

Iter 80/100: 1.620915518410499
Iter 90/100: 1.620001611709318
Iter 100/100: 1.61980197481781
BO iteration:  56
Iter 10/100: 2.3244866693136217
Iter 20/100: 1.974901104256574
Iter 30/100: 1.7751276571726977
Iter 40/100: 1.702167149919091
Iter 50/100: 1.6456316969585514
Iter 60/100: 1.6240041549445152
Iter 70/100: 1.6191980375740156
Iter 80/100: 1.6163967246891444
Iter 90/100: 1.615500783806594
Iter 100/100: 1.6153095418556302
BO iteration:  57
Iter 10/100: 2.3112028953506156
Iter 20/100: 1.965817861313833
Iter 30/100: 1.7681616586287574
Iter 40/100: 1.6966751885486646
Iter 50/100: 1.6409325684883234
Iter 60/100: 1.6194113143551918
Iter 70/100: 1.6146865967441146
Iter 80/100: 1.6119674454345134
Iter 90/100: 1.6110869185854264
Iter 100/100: 1.6109030079856865
BO iteration:  58
Iter 10/100: 2.2983765343998015
Iter 20/100: 1.9570469406092275
Iter 30/100: 1.7614442331235844
Iter 40/100: 1.6914347287664275
Iter 50/100: 1.636545293581464
Iter 60/100: 1.6151631383973013
Iter 70/100: 1.610501579

Iter 20/100: 1.8220148408024843
Iter 30/100: 1.6610659869574829
Iter 40/100: 1.6150379241300574
Iter 50/100: 1.574541001455533
Iter 60/100: 1.556268525866151
Iter 70/100: 1.5514561130357913
Iter 80/100: 1.550130428933642
Iter 90/100: 1.549376437766843
Iter 100/100: 1.549164411671109
BO iteration:  81
Iter 10/100: 2.092371102586109
Iter 20/100: 1.8175366766811125
Iter 30/100: 1.6576203405761922
Iter 40/100: 1.612275881147057
Iter 50/100: 1.5721701686154839
Iter 60/100: 1.554065566096036
Iter 70/100: 1.5492706013769477
Iter 80/100: 1.5479583761623181
Iter 90/100: 1.5472111007228322
Iter 100/100: 1.5469956494255854
BO iteration:  82
Iter 10/100: 2.085984227163644
Iter 20/100: 1.8131669920537687
Iter 30/100: 1.654252380346242
Iter 40/100: 1.6095565323181655
Iter 50/100: 1.5698099133018069
Iter 60/100: 1.5518468730433852
Iter 70/100: 1.5470563494969771
Iter 80/100: 1.545747836123981
Iter 90/100: 1.545002303787259
Iter 100/100: 1.544781890649416
BO iteration:  83
Iter 10/100: 2.0797677086788

Iter 90/100: 1.502312399574452
Iter 100/100: 1.5021042569780845
BO iteration:  105
Iter 10/100: 1.9708859781597707
Iter 20/100: 1.7335849481820564
Iter 30/100: 1.591246774094008
Iter 40/100: 1.556905680541708
Iter 50/100: 1.5224251008040932
Iter 60/100: 1.5068570864487454
Iter 70/100: 1.5028332283941013
Iter 80/100: 1.5018082829313268
Iter 90/100: 1.501169715840238
Iter 100/100: 1.5009586283626786
BO iteration:  106
Iter 10/100: 1.967086642635639
Iter 20/100: 1.7309899324201194
Iter 30/100: 1.5892563111089324
Iter 40/100: 1.5553175562930877
Iter 50/100: 1.5210714459546781
Iter 60/100: 1.5056101115109108
Iter 70/100: 1.5015969107596598
Iter 80/100: 1.5005765077007398
Iter 90/100: 1.4999427239265386
Iter 100/100: 1.4997297620531476
BO iteration:  107
Iter 10/100: 1.9634492029054396
Iter 20/100: 1.728582394394595
Iter 30/100: 1.5875912248757316
Iter 40/100: 1.5541376020182256
Iter 50/100: 1.5202202193149787
Iter 60/100: 1.5048780998565623
Iter 70/100: 1.5008290252403982
Iter 80/100: 1.499

...

Iter 10/100: 1.6776738345228497
Iter 20/100: 1.5443246090566374
Iter 30/100: 1.4693416505727521
Iter 40/100: 1.4679157216168803
Iter 50/100: 1.4581696932441925
Iter 60/100: 1.4557601770659918
Iter 70/100: 1.4546806864914432
Iter 80/100: 1.4540010393695968
Iter 90/100: 1.4538595872794824
Iter 100/100: 1.4536884993595751
BO iteration:  444
Iter 10/100: 1.6774570803981703
Iter 20/100: 1.5441764506700293
Iter 30/100: 1.4692278536380459
Iter 40/100: 1.4678201368662105
Iter 50/100: 1.4580839367201883
Iter 60/100: 1.455678769263313
Iter 70/100: 1.4546011567113295
Iter 80/100: 1.45392260970415
Iter 90/100: 1.4537818311634918
Iter 100/100: 1.453611318289218
BO iteration:  445
Iter 10/100: 1.677241288605945
Iter 20/100: 1.5440289391705144
Iter 30/100: 1.4691145308622147
Iter 40/100: 1.4677249426621923
Iter 50/100: 1.457998520586323
Iter 60/100: 1.4555976783297475
Iter 70/100: 1.4545219315896711
Iter 80/100: 1.4538444758360258
Iter 90/100: 1.4537043639817016
Iter 100/100: 1.453534419974005
BO ite

Iter 50/100: 1.4559235186357056
Iter 60/100: 1.45362442443604
Iter 70/100: 1.4525916542985609
Iter 80/100: 1.451938552133115
BO iteration:  472
Iter 10/100: 1.671777013361964
Iter 20/100: 1.5403043978937254
Iter 30/100: 1.4662695200841747
Iter 40/100: 1.4653287178287453
Iter 50/100: 1.4558478328416076
Iter 60/100: 1.453552592522112
Iter 70/100: 1.452521481340517
Iter 80/100: 1.451869348795416
BO iteration:  473
Iter 10/100: 1.6715856641658038
Iter 20/100: 1.5401732309735063
Iter 30/100: 1.4661679661591538
Iter 40/100: 1.465243143040342
Iter 50/100: 1.455770690348324
Iter 60/100: 1.453479137984412
Iter 70/100: 1.4524495593109519
Iter 80/100: 1.45179826683183
BO iteration:  474
Iter 10/100: 1.6713949237571493
Iter 20/100: 1.540042258717343
Iter 30/100: 1.4660660394206873
Iter 40/100: 1.4651570152998226
Iter 50/100: 1.4556927690470542
Iter 60/100: 1.4534047560228656
Iter 70/100: 1.4523766002018386
Iter 80/100: 1.451726044061017
BO iteration:  475
Iter 10/100: 1.6712059957860952
Iter 20/10

BO iteration:  529
Iter 10/100: 1.6620434101561081
Iter 20/100: 1.5336502196522488
Iter 30/100: 1.461156278163307
BO iteration:  530
Iter 10/100: 1.66189143948503
Iter 20/100: 1.5335464383740793
Iter 30/100: 1.4610767756429917
BO iteration:  531
Iter 10/100: 1.6617396184429263
Iter 20/100: 1.5334423409794762
Iter 30/100: 1.4609961179828734
BO iteration:  532
Iter 10/100: 1.6615886680004062
Iter 20/100: 1.5333391383773909
Iter 30/100: 1.4609168041129468
BO iteration:  533
Iter 10/100: 1.6614381550487562
Iter 20/100: 1.5332361277200248
Iter 30/100: 1.4608374875420391
BO iteration:  534
Iter 10/100: 1.6612881882612696
Iter 20/100: 1.533133454120772
Iter 30/100: 1.4607582675035746
BO iteration:  535
Iter 10/100: 1.6611391904094874
Iter 20/100: 1.533031840865854
Iter 30/100: 1.4606807215059174
BO iteration:  536
Iter 10/100: 1.6609906534451773
Iter 20/100: 1.532930450289208
Iter 30/100: 1.4606031480759931
BO iteration:  537
Iter 10/100: 1.660843092878306
Iter 20/100: 1.5328301318090214
Iter

Iter 30/100: 1.456164647787585
BO iteration:  601
Iter 10/100: 1.6523943535153343
Iter 20/100: 1.5270593931646876
Iter 30/100: 1.4561016603282866
BO iteration:  602
Iter 10/100: 1.6522761648494502
Iter 20/100: 1.526978233739744
Iter 30/100: 1.4560384597237623
BO iteration:  603
Iter 10/100: 1.6521583495442622
Iter 20/100: 1.5268973222726263
Iter 30/100: 1.4559754490840187
BO iteration:  604
Iter 10/100: 1.6520414605909708
Iter 20/100: 1.5268175947146592
Iter 30/100: 1.4559145815599521
BO iteration:  605
Iter 10/100: 1.6519248966236093
Iter 20/100: 1.5267380289644312
Iter 30/100: 1.4558537079683997
BO iteration:  606
Iter 10/100: 1.651808545089818
Iter 20/100: 1.5266584392223859
Iter 30/100: 1.455792447888873
BO iteration:  607
Iter 10/100: 1.6516929407126941
Iter 20/100: 1.5265797148516287
Iter 30/100: 1.45573262024628
elapsed_time:	 22.733149866263073  min.
total number of unique COFs acquired 605
oh no, top COF not acquired!


TypeError: slice indices must be integers or None or have an __index__ method